In [ ]:
import pandas as pd
import numpy as np
import feather
import datetime

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import geopandas as gpd

import PyNergy as pn

energy = pd.read_feather('data/energy.ftr')

sns.set_style = "seaborn-whitegrid"
sns.set_context("paper")


color = "#333"
bck_color = "#fff"
grid_color = "#ddd"

dark_mode = True
if dark_mode:
    color = "#eee"
    bck_color = "#333"
    grid_color = "#666"


charts_colors = {
    "blue_light": "#26ffff",
    "blue": "#2696ff",
    "blue_dark": "#0759a6",

    "pink_light": "#fdc2ff",
    "pink": "#f830ff",
    "pink_dark": "#830087",

    "orange_light": "#ffdc8c",
    "orange": "#ffc130",
    "orange_dark": "#de9a00",

    "yellow_light": "#fcffcc",
    "yellow": "#ffff00",
    "yellow_dark": "#8f8f09",

    "green_light": "#abffa6",
    "green": "#33ff26",
    "green_dark": "#157a0f",

    "red_light": "#ff7878",
    "red": "#ED0000",
    "red_dark": "#a11515"   
}
    

sns.set(
    rc={
        "axes.facecolor": bck_color,
        "grid.color": grid_color,
        "grid.linestyle": "-",
        "figure.facecolor": bck_color,
        "text.color": color,
        "xtick.color": color,
        "ytick.color": color,
        "axes.labelcolor": color,
        "axes.titlecolor": color,
        "axes.grid": True,
        'axes.spines.left': True,
        'axes.spines.bottom': True,
        'axes.spines.right': False,
        'axes.spines.top': False,
        "axes.edgecolor": grid_color,
    }
)


In [ ]:
energy.drop(columns=['pompage'], inplace=True)
consommation = energy[['region', 'date', 'consommation']]
consommation = consommation[ (consommation['date'].dt.year > 2012) & (consommation['date'].dt.year < 2021) ]

In [ ]:
conso_by_year = consommation.groupby(pd.Grouper(key='date', freq='Y'))['consommation'].sum().reset_index()

conso_by_year['consommation'] = conso_by_year['consommation'] / 1000

fig, ax = plt.subplots(figsize=(10, 6))
g = sns.barplot(
    x=conso_by_year['date'].dt.year, 
    y='consommation', data=conso_by_year, ax=ax, 
    color=charts_colors['yellow'], edgecolor=grid_color)

ax.set_title('Consommation selon les années en gW')
ax.set_xlabel('Années')
ax.set_ylabel('Consommation')

ax.set_yticks(np.arange(100, 1100, 100))
plt.savefig("outputs/03.1.1_-_Consommation_par_annee.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
conso_by_month = consommation.copy()

conso_by_month['m'] = conso_by_month['date'].dt.month
conso_by_month['mn'] = conso_by_month['date'].dt.month_name()
conso_by_month = conso_by_month.groupby(['m', 'mn'])['consommation'].mean().reset_index()
conso_by_month.sort_values(by='m', ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(
    x='mn', 
    y='consommation', 
    data=conso_by_month, ax=ax,
    color=charts_colors['yellow'], linewidth=4)

ax.set_title('Consommation selon les mois en gW')
ax.set_xlabel('Mois')
ax.set_ylabel('Consommation')

plt.setp(ax.get_xticklabels(), rotation=45)
plt.savefig("outputs/03.1.2_-_Consommation_par_mois.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
conso_by_dow = consommation.copy()
conso_by_dow['dow'] = conso_by_dow['date'].dt.dayofweek
conso_by_dow['dn'] = conso_by_dow['date'].dt.day_name()
conso_by_dow = conso_by_dow.groupby(['dn', 'dow'])['consommation'].mean().reset_index()
conso_by_dow.sort_values(by='dow', ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(x='dn', y='consommation', data=conso_by_dow, ax=ax, linewidth=4, color=charts_colors['yellow'])

ax.set_title('Consommation selon les jours de la semaine en mW')
ax.set_xlabel('Jours de la semaine')
ax.set_ylabel('Consommation')

plt.setp(ax.get_xticklabels(), rotation=45)
plt.savefig("outputs/03.1.3_-_Consommation_par_jour.png", dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
conso_by_hour = consommation.groupby(pd.Grouper(key='date', freq='H'))['consommation'].mean().reset_index()

month_less_conso = conso_by_hour[ conso_by_hour['date'].dt.month.isin([1, 12])].groupby(pd.Grouper(key='date', freq='H'))['consommation'].mean().reset_index()
month_most_conso = conso_by_hour[ conso_by_hour['date'].dt.month.isin([7, 8])].groupby(pd.Grouper(key='date', freq='H'))['consommation'].mean().reset_index()

fig, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(
    x=conso_by_hour['date'].dt.hour, y='consommation', data=conso_by_hour, ci=None,
    ax=ax, color=charts_colors['yellow'], drawstyle='steps-post', label='Moyenne annuelle')

sns.lineplot(
    x=month_less_conso['date'].dt.hour, y='consommation', data=month_less_conso, ci=None,
    ax=ax, color=charts_colors['blue_light'], alpha=0.5, drawstyle='steps-post', label='Moyenne hivernale')

sns.lineplot(
    x=month_most_conso['date'].dt.hour, y='consommation', data=month_most_conso, ci=None,
    ax=ax, color=charts_colors['pink'], alpha=0.5, drawstyle='steps-post', label='Moyenne estivale')

ax.set_title('Consommation selon les heures de la journée en mW')
ax.set_xlabel('Heures')
ax.set_ylabel('Consommation')

ax.set_xticks(np.arange(0, 24, 1))

plt.savefig("outputs/03.1.4_-_Consommation_par_heure.png", dpi=150, bbox_inches='tight')

plt.show()

In [ ]:
conso = energy[ (energy['date'].dt.year < 2021) & (energy['date'].dt.year > 2012) ]
conso = conso.groupby(['code_insee_region', 'region'])['consommation'].mean().reset_index()

gdf = pn.get_geometry(conso)

In [ ]:
c = mcolors.ColorConverter().to_rgb
cmap = pn.make_colormap(
    [c(charts_colors['yellow_light']), c(charts_colors['yellow'])])

fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot('consommation', edgecolor=bck_color, legend=True, cmap=cmap, legend_kwds={'shrink': 0.5}, ax=ax)
ax.set_title('Consommation')
ax.axis('off')
plt.savefig("outputs/03.1.5_-_Carte_consommation_par_region.png", dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
ech = energy[ (energy['date'].dt.year < 2021) & (energy['date'].dt.year > 2012) ]
ech = ech.groupby(['code_insee_region', 'region'])['ech_physiques'].mean().reset_index()

gdf = pn.get_geometry(ech)

fig, ax = plt.subplots(figsize=(10, 10))

gdf.plot('ech_physiques', edgecolor=bck_color, legend=True, cmap=cmap, legend_kwds={'shrink': 0.5}, ax=ax)
ax.set_title('Echanges Physiques')
ax.axis('off')
plt.savefig("outputs/03.1.6_-_Carte_echanges_physiques.png", dpi=150, bbox_inches='tight')

plt.show()